In [4]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="HDFCBANK")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 20                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=500_000,
    contract_size=1,
    sl_pct=0.0010,
    tp_pct=0.0025,
    trail_pct=0.0010,
    hold_max=10,
    upper=0.75,
    lower=0.35,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=HDFCBANK
[history] range UTC-naive: 2024-12-18 13:10:29.557223 → 2025-07-06 13:10:29.557223
[history] token=341249
[history] got 8751 bars, cursor→2025-03-28 13:12:00
[history] got 8171 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 13:10:29.557223, breaking loop
[history] complete 16922 bars 2024-12-18 13:09:00 → 2025-07-04 15:27:00
ret1 head: [0.0, 0.0008812271087488277, 0.000522767918558209, 0.00016499835001648755, 0.00010998075336821067]
ema_8 head: [1815.65, 1816.0055555555557, 1816.4932098765432, 1816.9391632373115, 1817.3304602956869]
ema_21 head: [1815.65, 1815.7954545454545, 1816.0140495867768, 1816.240045078888, 1816.4636773444436]
vwap head: [1815.8, 1816.2626918413723, 1816.7296524548292, 1817.014808392501, 1817.4580958579254]
vol_spike head: [0, 0, 0, 0, 0]
Training rows: 15,047
Testing  rows: 1,875  (need ≥ 60)

▶️ Pipeline 

In [3]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-06-23 11:27:00,2025-06-23 11:57:00,SELL,1947.3,1944.9000,TIME,118.177309,102,126.622691,200126.622691
2025-06-24 10:15:00,2025-06-24 10:45:00,SELL,1972.1,1969.4000,TIME,118.368668,101,154.331332,200280.954023
2025-06-24 11:42:00,2025-06-24 12:03:00,SELL,1975.9,1977.8759,SL,118.642367,101,-318.208267,199962.745756
2025-06-25 11:00:00,2025-06-25 11:30:00,SELL,1980.0,1976.7000,TIME,117.929279,100,212.070720,200174.816477


In [1]:
# ── Full Reload, Retrain & Back-test (fixed) ────────────────────────
import importlib, pandas as pd
import algo.features as feat_mod
import algo.model    as model_mod
importlib.reload(feat_mod); importlib.reload(model_mod)

from algo.features   import add_indicators, FEATURES
from algo.model      import load_or_train, predict_last, LOOKBACK
from algo.broker     import KiteWrapper
from algo.config     import load_config
from algo.backtester import backtest

# 1️⃣ Pull 200 days of 3-min history and build features
cfg    = load_config()
broker = KiteWrapper(cfg)
hist_all = broker.history(days=200, interval="3minute",
                          tradingsymbol="IDEA")
df_all   = add_indicators(hist_all, debug=True).ffill()

# 2️⃣ Split by *calendar days* instead of row count
n_test_days = 5                          # last 20 trading days ≈ 1500 bars
cutoff      = df_all.index[-1] - pd.Timedelta(days=n_test_days)

df_tr = df_all[df_all.index <  cutoff]    # ~180 days
df_te = df_all[df_all.index >= cutoff]    #  last 20 days  (≳ LOOKBACK rows)

print(f"Training rows: {len(df_tr):,}")
print(f"Testing  rows: {len(df_te):,}  (need ≥ {LOOKBACK})")

# 3️⃣ (Re)train the model
model = load_or_train(df_tr, retrain=False)

# 4️⃣ Inspect the pipeline
print("\n▶️ Pipeline steps:", model.named_steps)
gb = model.named_steps["gb"]
print("▶️ Final estimator:", gb)
print("▶️ n_features_in_:", gb.n_features_in_)
print(f"▶️ Expected      : {len(FEATURES)} × {LOOKBACK} = "
      f"{len(FEATURES)*LOOKBACK}\n")

# 5️⃣ Run back-test
trades, metrics = backtest(
    df_te,
    model=model,
    capital=200_000,
    contract_size=1,
    sl_pct=0.0030,
    tp_pct=0.0065,
    trail_pct=0.0020,
    hold_max=10,
    upper=0.486,
    lower=0.596,
    predict_fn=predict_last,
    debug=True,
)

# 6️⃣ Review results
print("Back-test metrics:", metrics)
print("\nFirst few trades:\n", trades.head())


[broker] loaded from /Users/sreejit/PycharmProjects/zerodha-bot/algo/broker.py
[KiteWrapper] initialized: symbol=RELIANCE on exch=NSE
[history] start: days=200, interval=3minute, symbol=IDEA
[history] range UTC-naive: 2024-12-18 13:11:45.051515 → 2025-07-06 13:11:45.051515
[history] token=3677697
[history] got 8750 bars, cursor→2025-03-28 13:12:00
[history] got 8171 bars, cursor→2025-07-04 15:30:00
[history] empty data for 2025-07-04 15:30:00->2025-07-06 13:11:45.051515, breaking loop
[history] complete 16921 bars 2024-12-18 13:12:00 → 2025-07-04 15:27:00
ret1 head: [0.0, 0.0, 0.001293661060802087, -0.0012919896640826156, 0.0]
ema_8 head: [7.73, 7.73, 7.732222222222223, 7.7317283950617295, 7.731344307270234]
ema_21 head: [7.73, 7.73, 7.730909090909091, 7.730826446280992, 7.7307513148009015]
vwap head: [7.730000000000001, 7.7252230708936835, 7.727830622932173, 7.728579430024545, 7.728689011564924]
vol_spike head: [0, 0, 0, 0, 0]
Training rows: 16,296
Testing  rows: 625  (need ≥ 60)

▶️ 

In [2]:
trades

,exit_ts,side,entry_price,exit_price,exit_reason,fees,qty,pnl,equity
entry_ts,,,,,,,,,
2025-06-30 12:12:00,2025-06-30 12:39:00,SELL,7.47,7.421445,TP,118.362470,26773,1181.600545,201181.600545
2025-06-30 12:42:00,2025-06-30 13:12:00,SELL,7.43,7.420000,TIME,119.081974,27076,151.678026,201333.278571
2025-06-30 13:15:00,2025-06-30 13:45:00,SELL,7.42,7.410000,TIME,119.136244,27133,152.193756,201485.472326
2025-06-30 13:48:00,2025-06-30 14:09:00,SELL,7.41,7.432230,SL,119.445892,27191,-723.901822,200761.570504
2025-06-30 14:12:00,2025-06-30 14:42:00,SELL,7.43,7.450000,TIME,119.167446,27020,-659.567446,200102.003058
...,...,...,...,...,...,...,...,...,...
2025-07-04 13:45:00,2025-07-04 13:57:00,SELL,7.33,7.351990,SL,120.190320,27771,-730.874610,202833.605753
2025-07-04 14:00:00,2025-07-04 14:30:00,SELL,7.35,7.340000,TIME,119.672795,27596,156.287205,202989.892958
2025-07-04 14:33:00,2025-07-04 14:42:00,SELL,7.35,7.372050,SL,119.983613,27617,-728.938463,202260.954495
